In [21]:
import torch_geometric
import torch
from torch_geometric.datasets import OGB_MAG
import torch_geometric.transforms as T
from torch_geometric.loader import NeighborLoader

In [22]:
torch.__version__

'1.8.2'

In [23]:
torch_geometric.__version__

'2.0.1'

In [24]:
dataset = OGB_MAG(root='./data', preprocess='transe', transform=T.ToUndirected(merge=True))
data = dataset[0]
print(data)

HeteroData(
  paper={
    x=[736389, 128],
    y=[736389],
    train_mask=[736389],
    val_mask=[736389],
    test_mask=[736389]
  },
  author={ x=[1134649, 256] },
  field_of_study={ x=[59965, 256] },
  institution={ x=[8740, 256] },
  (author, affiliated_with, institution)={ edge_index=[2, 1043998] },
  (author, writes, paper)={ edge_index=[2, 7145660] },
  (paper, cites, paper)={ edge_index=[2, 10792672] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 7505078] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 1043998] },
  (paper, rev_writes, author)={ edge_index=[2, 7145660] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 7505078] }
)


In [28]:
transe_emb = torch.load('./data/mag/raw/mag_transe_emb.pt', map_location='cpu')
print(transe_emb['paper'].shape)
print(transe_emb['author'].shape)

torch.Size([736389, 256])
torch.Size([1134649, 256])


In [26]:
data.x_dict['paper'].shape

torch.Size([736389, 128])

In [27]:
# x_dict = {key: torch.cat([x, transe_emb[key]], dim=1) for key, x in data.x_dict.items()}
# data.x_dict = x_dict
data['paper'].x = torch.cat([data['paper'].x, transe_emb['paper']], dim=1)
data['author'].x = torch.cat([data['author'].x, transe_emb['author']], dim=1)
data['field_of_study'].x = torch.cat([data['field_of_study'].x, transe_emb['field_of_study']], dim=1)
data['institution'].x = torch.cat([data['institution'].x, transe_emb['institution']], dim=1)
print(data.x_dict['paper'].shape)
print(data.x_dict['author'].shape)
print(data.x_dict['field_of_study'].shape)
print(data.x_dict['institution'].shape)

torch.Size([736389, 384])
torch.Size([1134649, 512])
torch.Size([59965, 512])
torch.Size([8740, 512])


In [15]:
data.x_dict = {key: torch.cat([x, transe_emb[key]], dim=1) for key, x in data.x_dict.items()}
data.x_dict['paper'].shape

torch.Size([736389, 128])

In [29]:
data.metadata()

(['paper', 'author', 'field_of_study', 'institution'],
 [('author', 'affiliated_with', 'institution'),
  ('author', 'writes', 'paper'),
  ('paper', 'cites', 'paper'),
  ('paper', 'has_topic', 'field_of_study'),
  ('institution', 'rev_affiliated_with', 'author'),
  ('paper', 'rev_writes', 'author'),
  ('field_of_study', 'rev_has_topic', 'paper')])

In [30]:
data.x_dict

{'paper': tensor([[-0.0954,  0.0408, -0.2109,  ...,  0.7162,  0.4707,  0.7670],
         [-0.1510, -0.1073, -0.2220,  ...,  0.6170,  0.4240, -0.1924],
         [-0.1148, -0.1760, -0.2606,  ...,  0.6009, -0.2398,  0.3472],
         ...,
         [ 0.0228, -0.0865,  0.0981,  ...,  0.4428, -0.6617,  0.9344],
         [-0.2891, -0.2029, -0.1525,  ..., -0.1030, -0.5050,  0.6917],
         [-0.0890, -0.0348, -0.2642,  ..., -0.3364, -0.2887, -0.3819]]),
 'author': tensor([[ 0.3226,  0.0584, -0.3172,  ..., -0.1393,  0.3022, -0.8178],
         [ 0.0438, -0.1213,  0.1804,  ..., -0.3374,  0.4846, -0.4272],
         [-0.2300, -0.3192,  0.1979,  ..., -0.1007,  1.3342, -0.4569],
         ...,
         [-0.5160,  0.1867,  0.2854,  ..., -0.1987, -0.6515,  0.5417],
         [-0.3726, -0.1563, -0.1685,  ...,  0.1218,  0.4354, -0.0998],
         [-0.7412,  0.5156,  0.0240,  ..., -0.2002, -0.2625, -0.1643]]),
 'field_of_study': tensor([[-0.5958,  0.4260,  0.0320,  ..., -0.2224,  0.2888, -0.3681],
        

In [31]:
data.edge_index_dict

{('author',
  'affiliated_with',
  'institution'): tensor([[      0,       1,       2,  ..., 1134645, 1134647, 1134648],
         [    845,     996,    3197,  ...,    5189,    4668,    4668]]),
 ('author',
  'writes',
  'paper'): tensor([[      0,       0,       0,  ..., 1134647, 1134648, 1134648],
         [  19703,  289285,  311768,  ...,  657395,  671118,  719594]]),
 ('paper',
  'cites',
  'paper'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [    88,  27449, 121051,  ..., 707740, 732008, 732389]]),
 ('paper',
  'has_topic',
  'field_of_study'): tensor([[     0,      0,      0,  ..., 736388, 736388, 736388],
         [   145,   2215,   3205,  ...,  21458,  22283,  31934]]),
 ('institution',
  'rev_affiliated_with',
  'author'): tensor([[    845,     996,    3197,  ...,    5189,    4668,    4668],
         [      0,       1,       2,  ..., 1134645, 1134647, 1134648]]),
 ('paper',
  'rev_writes',
  'author'): tensor([[  19703,  289285,  311768,  ...,  657

In [32]:
train_loader = NeighborLoader(data, num_neighbors=[15, 15], batch_size=128, input_nodes=('paper', data['paper'].train_mask), shuffle=True)

In [33]:
print(train_loader)
batch = next(iter(train_loader))
print(batch)

NeighborLoader()
HeteroData(
  paper={
    x=[26117, 384],
    y=[26117],
    train_mask=[26117],
    val_mask=[26117],
    test_mask=[26117],
    batch_size=128
  },
  author={ x=[4997, 512] },
  field_of_study={ x=[3319, 512] },
  institution={ x=[369, 512] },
  (author, affiliated_with, institution)={ edge_index=[2, 0] },
  (author, writes, paper)={ edge_index=[2, 6525] },
  (paper, cites, paper)={ edge_index=[2, 15905] },
  (paper, has_topic, field_of_study)={ edge_index=[2, 11942] },
  (institution, rev_affiliated_with, author)={ edge_index=[2, 875] },
  (paper, rev_writes, author)={ edge_index=[2, 5405] },
  (field_of_study, rev_has_topic, paper)={ edge_index=[2, 13461] }
)


In [39]:
from torch_geometric.nn import TransformerConv, GCNConv, GATConv, SAGEConv, to_hetero, Linear
import torch.nn as nn
import torch.nn.functional as F 

In [40]:
class Net1(torch.nn.Module):
    def __init__(self, hidden_dim, num_classes, num_layers=2) -> None:
        super().__init__()

        self.hidden_dim = hidden_dim
        self.num_classes = num_classes
        self.num_layers = num_layers

        self.convs = nn.ModuleList()
        self.lins = nn.ModuleList()
        self.bns = nn.ModuleList()

        for i in range(self.num_layers):
            self.convs.append(GATConv((-1, -1), hidden_dim))
            self.lins.append(Linear(-1, hidden_dim))
            self.bns.append(nn.BatchNorm1d(hidden_dim))

        # self.dropout = torch.nn.Dropout()
        self.fc_out = Linear(-1, num_classes)

    def forward(self, x, edge_index):
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index) + self.lins[i](x)
            x = self.bns[i](x)
            x = F.relu(x)
            x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc_out(x)
        return x

In [42]:
device = torch.device('cuda:2' if torch.cuda.is_available() else 'cpu')
print(device)
model = Net1(hidden_dim=64, num_classes=dataset.num_classes, num_layers=2)
model = to_hetero(model, data.metadata(), aggr='sum').to(device)
print(model)

cuda:2
GraphModule(
  (convs): ModuleList(
    (0): ModuleDict(
      (author__affiliated_with__institution): GATConv((-1, -1), 64, heads=1)
      (author__writes__paper): GATConv((-1, -1), 64, heads=1)
      (paper__cites__paper): GATConv((-1, -1), 64, heads=1)
      (paper__has_topic__field_of_study): GATConv((-1, -1), 64, heads=1)
      (institution__rev_affiliated_with__author): GATConv((-1, -1), 64, heads=1)
      (paper__rev_writes__author): GATConv((-1, -1), 64, heads=1)
      (field_of_study__rev_has_topic__paper): GATConv((-1, -1), 64, heads=1)
    )
    (1): ModuleDict(
      (author__affiliated_with__institution): GATConv((-1, -1), 64, heads=1)
      (author__writes__paper): GATConv((-1, -1), 64, heads=1)
      (paper__cites__paper): GATConv((-1, -1), 64, heads=1)
      (paper__has_topic__field_of_study): GATConv((-1, -1), 64, heads=1)
      (institution__rev_affiliated_with__author): GATConv((-1, -1), 64, heads=1)
      (paper__rev_writes__author): GATConv((-1, -1), 64, hea

In [43]:
@torch.no_grad()
def init_params():
    # Initialize lazy parameters via forwarding a single batch to the model:
    batch = next(iter(train_loader))
    batch = batch.to(device)
    model(batch.x_dict, batch.edge_index_dict)

In [44]:
import torch

In [46]:
emb = torch.load('./data/mag/raw/mag_metapath2vec_emb.pt', map_location='cpu')
print(emb['paper'].shape)
print(emb['author'].shape)

torch.Size([736389, 128])
torch.Size([1134649, 128])


In [7]:
emb['author'].shape

torch.Size([1134649, 128])

In [9]:
d1 = {'1': torch.rand((2, 3)), '2': torch.rand((4, 3))}
d2 = {'1': torch.rand((2, 3)), '2': torch.rand((4, 3))}
{key: x + d2[key] for key, x in d1.items()}

{'1': tensor([[1.8597, 0.6279, 1.0817],
         [0.5914, 0.7982, 0.2904]]),
 '2': tensor([[0.7516, 1.5576, 1.0026],
         [0.9337, 1.3418, 1.0792],
         [0.8797, 0.9528, 0.7635],
         [1.3031, 0.7651, 1.3356]])}